In [25]:
import numpy as np
import pandas as pd
import joblib
import datetime
import matplotlib.pyplot as plt
from sklearn.tree import ExtraTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hyperopt import hp, anneal, Trials, STATUS_OK
from hyperopt.fmin import fmin
from functools import partial

In [3]:
!pip install hyperopt

In [26]:
# Load the datasets
train_df = pd.read_excel("train_data.xlsx")
test_df = pd.read_excel("test_data.xlsx")

In [27]:
# Define features (X) and target (y) for training and testing
X_train = train_df.drop(['P'], axis=1)
y_train = train_df['P']
X_test = test_df.drop(['P'], axis=1)
y_test = test_df['P']


In [17]:
def model_metrics(model, X, y):
    """Compute R2 score for model evaluation."""
    y_pred = model.predict(X)
    return r2_score(y, y_pred)

def bayes_fmin(X_train, X_test, y_train, y_test, eval_iters=100):
    """Hyperparameter optimization using Bayesian search."""
    def objective(params):
        model = ExtraTreeRegressor(
            max_depth=int(params['max_depth']),
            max_features=int(params['max_features']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42
        )
        model.fit(X_train, y_train)
        loss = -model_metrics(model, X_test, y_test)
        return {"loss": loss, "status": STATUS_OK}

    space = {
        'max_depth': hp.quniform('max_depth', 1, 50, 1),
        'max_features': hp.quniform('max_features', 1, 3, 1),
        'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
        'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 30, 1),
    }
    best_params = fmin(objective, space, algo=partial(anneal.suggest), max_evals=eval_iters, trials=Trials())
    best_params = {key: int(value) for key, value in best_params.items()}
    return best_params

In [18]:
# Hyperparameter tuning
best_params = bayes_fmin(X_train, X_test, y_train, y_test, 500)
print("Best Hyperparameters:", best_params)

100%|██████████| 500/500 [00:06<00:00, 76.65trial/s, best loss: -0.9159689880054149]  
Best Hyperparameters: {'max_depth': 18, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 10}


In [19]:
# Train ExtraTreeRegressor model
model = ExtraTreeRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)


ExtraTreeRegressor(max_depth=18, max_features=2, min_samples_split=10,
                   random_state=42)

In [20]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [21]:

# Model Evaluation
metrics = {
    "RMSE_Train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
    "MAE_Train": mean_absolute_error(y_train, y_train_pred),
    "R2_Train": r2_score(y_train, y_train_pred),
    "RMSE_Test": np.sqrt(mean_squared_error(y_test, y_test_pred)),
    "MAE_Test": mean_absolute_error(y_test, y_test_pred),
    "R2_Test": r2_score(y_test, y_test_pred)
}

for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

RMSE_Train: 35.5158
MAE_Train: 21.7307
R2_Train: 0.9578
RMSE_Test: 47.2356
MAE_Test: 28.3384
R2_Test: 0.9160
